In [1]:
import pandas as pd 
import numpy as np

# Let's change how printing the series works. I need to see all elements in the Series 
# source: http://stackoverflow.com/questions/19124601/is-there-a-way-to-pretty-print-the-entire-pandas-series-dataframe
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 70)

jobs = pd.read_csv('../data/stackoverflow_jobs.csv')

# Exploring data

In [2]:
jobs.columns

Index([u'jobid', u'date', u'description', u'employer', u'location', u'salary',
       u'tags', u'title', u'url', u'weeknum'],
      dtype='object')

In [3]:
jobs.dtypes

jobid           int64
date           object
description    object
employer       object
location       object
salary         object
tags           object
title          object
url            object
weeknum         int64
dtype: object

In [4]:
jobs.describe()

,jobid,weeknum
count,20032.000000,20032.000000
mean,117387.751448,33.072983
std,11539.370222,5.918207
min,15018.000000,22.000000
25%,115874.750000,29.000000
50%,120549.000000,33.000000
75%,123527.000000,38.000000
max,127431.000000,42.000000


In [5]:
jobs.head()

,jobid,date,description,employer,location,salary,tags,title,url,weeknum
0,113232,2016-07-25,Postmates runs the largest on-demand delivery ...,Postmates Inc.,"San Francisco, CA","$120,000 - 170,000\r\n\r\n ...","[ios, objective-c, swift]",Software Engineer: iOS,/jobs/113232/software-engineer-ios-postmates-inc,30
1,121583,2016-07-25,racking CodeCRCCHSPBKRSFDCA1162Job Description...,Health Dialog,"Bakersfield, CA",NaN,[cold-fusion],Care Coach - Spanish Bilingual,/jobs/121583/care-coach-spanish-bilingual-heal...,30
2,121582,2016-07-25,You are an experienced Drupal developer and we...,C4Media,"Sibiu, Romania",NaN,"[ubuntu-lamp, drupal]",Drupal Developer,/jobs/121582/drupal-developer-c4media,30
3,109825,2016-07-25,Summary\r\nAs Spot Trading continues to assert...,Spot Trading LLC,"Chicago, IL",NaN,"[database, sql-server, nosql, cassandra, mongo...",Database Administrator - DevOps - SQL,/jobs/109825/database-administrator-devops-sql...,30
4,105355,2016-07-25,The primary purpose of the Quality Assurance E...,TeleTracking Technologies,"Pittsburgh, PA",NaN,"[c#, asp.net, qa]",QA Engineer,/jobs/105355/qa-engineer-teletracking-technolo...,30


In [6]:
jobs.tail()

,jobid,date,description,employer,location,salary,tags,title,url,weeknum
20027,123217,2016-10-18,"At INFARE, we are not happy if we simply achie...",INFARE,"Copenhagen, Denmark",NaN,"[""java"",""scala"",""c#"","".net"",""bigdata""]",Senior Back End Developer - Big Data,/jobs/123217/senior-back-end-developer-big-dat...,42
20028,111740,2016-10-18,Who we are:\r\n\r\nWe implement GSMA standards...,Samsung R&D Institute,"Warsaw, Poland",NaN,"[""c"",""c++"",""git"",""tcp"",""rcs""]",Developer C/C++ (Network 2020),/jobs/111740/developer-c-c-plus-plus-network-2...,42
20029,116619,2016-10-18,We are looking for cutting-edge QA talents who...,Project A Ventures,"Berlin, Deutschland",NaN,"[""automated-tests"",""performance-testing"",""jira...",QA Automation Engineer (m/f) wanted!,/jobs/116619/qa-automation-engineer-m-f-wanted...,42
20030,104969,2016-10-18,As a member of Splash Damage’s AI Programming ...,Splash Damage,"Bromley, UK",NaN,"[""c++"",""unreal-engine4"",""artificial-intelligen...",AI Programmer,/jobs/104969/ai-programmer-splash-damage,42
20031,123218,2016-10-18,Are you passionate about improving conversion ...,simplesurance GmbH,"Berlin, Germany",NaN,"[""html"",""css"",""adobe""]",Conversion Optimization Specialist (f/m),/jobs/123218/conversion-optimization-specialis...,42


# Salary


## Replacing NA values with empty strings in the salary column

In [7]:
jobs.salary = jobs.salary.fillna('')

## Extracting equity  

In [8]:
jobs['equity'] = jobs['salary'].str.contains('Provides Equity')

## Extracting currency and high - low salary


Need to extract currency, salary_low and salary_high from salary field and copy it to their own columns.

Using regex here to capture parts of the salary field into three columns: 
    - currency will capture zero or more characters that are non digits
    - number_low captures one or more characters that are digits and spearators (currently only comma is used)
    - number high will capture all the numbers plus separators from the dash until the end of the string 

In [9]:
# salary = jobs.salary
salary = jobs.salary.map(lambda x: x.replace("Provides Equity","").replace("/","").strip())

sal = salary.str.extract('(?P<currency>[^\d]*)(?P<number_low>[\d,]+) - (?P<number_high>[\d,]+$)')

sal.number_low = sal.number_low.fillna(0)
sal.number_high = sal.number_high.fillna(0)
sal.currency = sal.currency.fillna('')

# mapping the new columns back
jobs['currency'] = sal.currency
jobs['salary_low'] = sal.number_low
jobs['salary_high'] = sal.number_high


# Location 

We need better location information, so we can do analysis by countries and cities. For this we need to extract country, state and city out of location column. But first let's remove the __na__ values from location column. 

Then use a lambda to split the location into individual fields.  

In [10]:
jobs.location = jobs.location.fillna('') # sometimes we have nothing in the location field. 

location_split = lambda x: pd.Series([i for i in x.split(',')])
locations = jobs['location'].apply(location_split)

locations.rename(columns={0:'city', 1: 'location_1', 2: 'location_2'},inplace=True)

## Fixing US locations

US locations seems to be special. They are in the form of _city, state_, we need this to be in form of _city, state, country_, so let's fix this first. 

If we have a US state in _location_1_ column then put _US_ in _location_2_. 


In [11]:
# Fixing US States
us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

locations['location_1'] = locations['location_1'].str.strip()
locations.loc[locations['location_1'].isin(us_states),'location_2'] = "US"

## Filling the state and country columns

If in a row location_2 is null then location_1 contains the country of that location, if location_2 is not empty thne location_2 is going to be the country and location_1 will contain the state. 

In [12]:
# if location_2 is null then location_1 column has the country 
# if location_2 is not null then location_2 has the country and location_1 contains the state 
jobs['country'] = np.where(locations['location_2'].isnull(), locations['location_1'], locations['location_2'])
jobs['state'] = np.where(locations['location_2'].notnull(), locations['location_1'], '')

jobs['city'] = locations['city']

# filling na for country 
jobs.country = jobs.country.fillna('')

# stripping spaces from new columns
jobs['city'] = jobs['city'].str.strip()
jobs['country'] = jobs['country'].str.strip()

Now we can see what countries are posting the most jobs. It seems that the US, Deutschland, Germany and the UK are the top countries. But wait. Aren't Germany and Deutschland are the same country? Let's fix this and some other countries with native names. 

In [13]:
# replacing some of the country names with their english version 
jobs.loc[jobs['country'].str.contains('Deutschland'),'country'] = 'Germany' # Deutschland -> Germany
jobs.loc[jobs['country'].str.contains('Österreich'),'country'] = 'Austria' # Österreich -> Austria
jobs.loc[jobs['country'].str.contains('Suisse'), 'country'] = 'Switzerland' # Suisse -> Switzerland
jobs.loc[jobs['country'].str.contains('Schweiz'), 'country'] = 'Switzerland' # Schweiz -> Switzerland
jobs.loc[jobs['country'].str.contains('Espagne'), 'country'] = 'Spain' # Espagne -> Spain
jobs.loc[jobs['country'].str.contains('République tchèque'), 'country'] = 'Czech Republic' # République tchèque -> Czech Republic
jobs.loc[jobs['country'].str.contains('Niederlande'), 'country'] = 'Netherlands' # Niederlande -> Netherlands

jobs['country'].value_counts().head()

US             9262
Germany        3260
UK             1916
Netherlands     915
Canada          665
Name: country, dtype: int64

In [14]:
jobs['city'].value_counts()

New York                      1070
London                        1012
Berlin                         991
San Francisco                  761
München                        412
Amsterdam                      403
Seattle                        398
Philadelphia                   350
Chicago                        329
Toronto                        301
No office location             284
Hamburg                        282
Washington                     248
Boston                         208
Dublin                         187
Cambridge                      185
Austin                         181
Stockholm                      174
Los Angeles                    173
Denver                         170
Sydney                         147
Singapore                      137
Atlanta                        131
Rotterdam                      130
Mountain View                  130
Düsseldorf                     124
Bellevue                       121
Helsinki                       119
Zürich              